# Colour Correction using Splines

### Define constants

In [8]:
%load_ext autoreload
%autoreload 2
import colour
from data import load_dataset_sfu, load_dataset_csv, load_illuminant_cmfs, load_camera, load_insitu
import numpy as np

RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)

SFU_FILE_PATH = 'reflect_db.reflect'
CAVE_FOSTER2004_PATH = 'cave.csv'
FOSTER_50_PATH = 'foster50.csv'
INSITU_PATH = "insitu_dataset.csv"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'load_illuminant_cmfs' from 'data' (c:\Users\jonis\Koulu\Dippa\CameraCalibration-main\data.py)

In [ ]:
sfu_dataset = load_dataset_sfu(SFU_FILE_PATH)
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)

TRAIN = cave_foster2004_dataset
TEST = foster_50_dataset
VALIDATION = sfu_dataset
CAMERA = 'nikon'

### Spectral sensitivities of various cameras

In [ ]:

MSDS_TRAIN, SDS_TRAIN = load_camera(CAMERA)


### Computing Observer Responses
We can easily change the order of test and train sets here

In [ ]:
illuminant, cmfs = load_illuminant_cmfs()
colour.plotting.plot_multi_cmfs([MSDS_TRAIN, cmfs])
response_trainset_xyz = colour.characterisation.training_data_sds_to_XYZ(TRAIN, cmfs, illuminant)
response_testset_xyz = colour.characterisation.training_data_sds_to_XYZ(TEST, cmfs, illuminant)
response_validation_xyz = colour.characterisation.training_data_sds_to_XYZ(VALIDATION, cmfs, illuminant)

### Computing Camera Responses

In [ ]:
response_trainset_camera, _ = colour.characterisation.training_data_sds_to_RGB(TRAIN, MSDS_TRAIN, SDS_TRAIN)
response_testset_camera, _ = colour.characterisation.training_data_sds_to_RGB(TEST, MSDS_TRAIN, SDS_TRAIN)
response_validationset_camera,  _ = colour.characterisation.training_data_sds_to_RGB(VALIDATION, MSDS_TRAIN, SDS_TRAIN)

In [ ]:
response_trainset_camera_augmented = np.vstack((response_trainset_camera, response_trainset_camera / 2))
response_trainset_xyz_augmented = np.vstack((response_trainset_xyz, response_trainset_xyz / 2))


### Fit Generalized Additive Model with P-splines

In [ ]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
from skopt import BayesSearchCV

np.int = np.int64
custom_scorer = make_scorer(deltae_mean, greater_is_better=False)

param_grid_gam = {
    'lams': [1*10**(-i) for i in range(2, 10)],
}
print(param_grid_gam)
gam = GAMOptimizer(lams=0.0001,order=3, n_splines=5)

bayes_search = BayesSearchCV(
    gam,
    param_grid_gam,
    n_iter=50,  # Number of iterations for the Bayesian optimization
    cv=5,
    n_jobs=-1,
    scoring=custom_scorer,
    verbose=2,
    random_state=RANDOM_STATE,  # Set a random seed for reproducibility
)
bayes_search.fit(response_trainset_camera, response_trainset_xyz)

print("Best parameters:", bayes_search.best_params_)



pred(bayes_search, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")